<a href="https://colab.research.google.com/github/rohdimp24/EVA8/blob/master/EVA4_Session_2_Rohit_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
from __future__ import print_function
## importing pytroch
import torch
## the neural network layer is provided by pytorch.nn
import torch.nn as nn
## for activation functions we use functional layer 
import torch.nn.functional as F
## provides optimizer like adam, SGD
import torch.optim as optim
## this is to fetch the MNIST data set and also to apply some transformations
from torchvision import datasets, transforms

In [44]:
## this class defines the neural network.
## esentially it has the definition of the layers, kernel sizes and the forward pass
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)         # input - 1x28x28    | output - 32x28x28     | RF - 3x3  
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)        # input - 32x28x28   | output - 64x28x28     | RF - 5x5  
        self.pool1 = nn.MaxPool2d(2, 2)                     # input - 64x28x28   | output - 64x14x14     | RF - 10x10  
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)       # input - 64x14x14   | output - 128x14x14    | RF - 12x12  
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)      # input - 128x14x14  | output - 256x14x14    | RF - 14x14  
        self.pool2 = nn.MaxPool2d(2, 2)                     # input - 256x14x14  | output - 256x7x7      | RF - 28x28  
        self.conv5 = nn.Conv2d(256, 512, 3)                 # input - 256x7x7    | output - 512x5x5      | RF - 30x30  
        self.conv6 = nn.Conv2d(512, 1024, 3)                # input - 512x5x5    | output - 1024x3x3     | RF - 32x32  
        self.conv7 = nn.Conv2d(1024, 10, 3)                 # input - 1024x3x3   | output - 10x1x1       | RF - 34x34  

    def forward(self, x):
        ## if there is padding the size will remain the same after convolution operation
        ## SIZE: image(x) (CONV1)-> 28x28  (CONV2)-> 28x28  (MAXPOOL)->14x14 
        ## RF: image(x) (CONV1)-> 3x3  (CONV2)-> 5x5  (MAXPOOL)->10x10 
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))

        ##SIZE: (CONV3)->14x14 (CONV4)->14x14  (MAXPOOL)->7x7  
        ##RF: (CONV3)->12x12 (CONV4)->14x14  (MAXPOOL)->28x28  
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        
        ##SIZE: (CONV5)->5x5 (CONV6)-> 3x3
        ##RF: (CONV5)->30x30 (CONV6)-> 32x32
        x = F.relu(self.conv6(F.relu(self.conv5(x))))

        ##SIZE: (CONV7)->1x1
        ## RF: 34x34
        #x = F.relu(self.conv7(x))... cant have a activation function in the last layer as it will remove -ve gradients
        x = F.tanh(self.conv7(x)) 
        ## or we can even say 
        ## x=self.conv7(x)
        ## this will remove the extra dimantions [128,10,1,1] --> [128,10]
        x = x.view(-1, 10)
        ## softmax for probability
        return F.log_softmax(x)

In [45]:
#!pip install torchsummary
from torchsummary import summary
## check for the GPU
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
## this will send the model to the chosen device
model = Net().to(device)
## prints the architecture
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

<ipython-input-44-dd059e5dfda0>:39: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [46]:


torch.manual_seed(1)
## 128 images will be processed at a time.. there will be multiple batches of 128 imgaes rill all the images have been read
batch_size = 128

## the pin_memory is some GPU memory management
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

## reading the MNIST data set and normalizging it
##https://discuss.pytorch.org/t/normalization-in-the-mnist-example/457
## created a train & test data loader that is a standard way of provodong the data to the model in pytorch
## we can additionally perform some transformations to the data before actually passing the data which is done using the transforms
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [47]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    ## this will show the progress
    pbar = tqdm(train_loader)
    #loss=0
    ## read each batch that will have 128 images and 128 labels
    ## this is a standard training loop in pytorch
    #optimizer.zero_grad()
    for batch_idx, (data, target) in enumerate(pbar):
        ## move the data and the target to the same device as the model
        data, target = data.to(device), target.to(device)
        ## clears old gradients from the last step (otherwise you’d just accumulate the gradients from all loss.backward() calls) 
        optimizer.zero_grad()
        ## forward pass with the batch of the data
        output = model(data)
        ## calculate the loss for this batch
        loss = F.nll_loss(output, target)
        ## computes the derivative of the loss w.r.t. the parameters (or anything requiring gradients) using backpropagation.
        loss.backward()
        ## causes the optimizer to take a step based on the gradients of the parameters
        optimizer.step()
        ## show the loss for this batch
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        ##print("loss after the batch",f'loss={loss.item()} batch_id={batch_idx}' )
        ## why are we not adding up the training loss just like in the test??


def test(model, device, test_loader):
    ## we are using the model in evaluation mode...meaning there will be no trainign
    model.eval()
    test_loss = 0
    correct = 0
    ## no gradient calculation will happen
    with torch.no_grad():
        for data, target in test_loader:
            ## mode the data to the same device as the model
            data, target = data.to(device), target.to(device)
            ## get the result of the model
            output = model(data)
            ## calulate the nll loss ..this is the testing loss
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            ## using the argmax to find out which is the most probable class (number) identified
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            ## check predictions with the ground truth
            correct += pred.eq(target.view_as(pred)).sum().item()
    ##normalize the test loss by the size of the dataset
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [48]:
## sending the architure to the device (cpu or gpu)
model = Net().to(device)
## using the SGD optimizer with the learning rate of 0.01 
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
## number of epochs can be controlled here
for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    ## we dont need any optimizer and epoch for the testing phase
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-44-dd059e5dfda0>:39: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.862337052822113 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 27.75it/s]



Test set: Average loss: 0.8698, Accuracy: 9622/10000 (96%)

